## Install Required Libraries

In [ ]:
!pip install datasets -q
!pip install tokenizers -q
!pip install transformers -q
!pip install seqeval -q
!pip install hazm

     |████████████████████████████████| 365 kB 14.7 MB/s 
     |████████████████████████████████| 212 kB 71.1 MB/s 
     |████████████████████████████████| 101 kB 2.7 MB/s 
     |████████████████████████████████| 141 kB 60.4 MB/s 
     |████████████████████████████████| 596 kB 66.0 MB/s 
     |████████████████████████████████| 127 kB 69.9 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 6.6 MB 12.6 MB/s 
     |████████████████████████████████| 4.7 MB 15.2 MB/s 
     |████████████████████████████████| 43 kB 1.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 316 kB 15.3 MB/s 
     |████████████████████████████████| 233 kB 67.2 MB/s 

In [ ]:
import tqdm
from hazm import *
import copy
from datasets.dataset_dict import DatasetDict
from datasets import Dataset
import datasets

## Load dataset

In [ ]:
!gdown --id 1ymVdW0XnWHGSEGnFRnzKs9Em6t2PZSe6

In [ ]:
import json
f = open('dataset_annotated_splited.json')
dataset = json.load(f)

In [ ]:
data_train = dataset['train']
data_test = dataset['test']
data_eval = dataset['eval']

In [ ]:
def ner_maker(ner_start, ner_end, data_train, h_t, ner_tag):
  ner_len = len(word_tokenize(data_train[h_t][ner_start: ner_end]))
  return [f'B-{ner_tag}']+[f'I-{ner_tag}']*(ner_len-1)

In [ ]:
def o_maker(o_start, o_end, text):
  i = 0
  if '/' in text[o_start: o_end] or '-' in text[o_start: o_end]:
    i=1
  return ['O']*(len(word_tokenize(text[o_start: o_end]))-i)

In [ ]:
labels = ["O" , "B-PER" , "I-PER" , "B-LOC" , "I-LOC" , "B-NAT", "I-NAT", "B-EVE", "I-EVE", "B-TIM", "I-TIM", "B-DAT", "I-DAT", "B-ORG", "I-ORG", "B-mainLOC", "I-mainLOC"]

In [ ]:
#######################################TRAIN########################################
text = []
for i in range(len(data_train)):
  text.append(data_train[i]['header'])
  text.append(data_train[i]['text'])

text_tokens = [word_tokenize(sent) for sent in tqdm.tqdm_notebook(text)]

langs = copy.deepcopy(text_tokens)
ner_tags = copy.deepcopy(text_tokens)

for i , x in enumerate(text_tokens):
  for j , y in enumerate(x):
    langs[i][j]='fa'

spans=[]
for i in range(len(data_train)):
    ls = []
    for j in range(len(data_train[i]['annotations'][0]['header'])):
        s = data_train[i]['annotations'][0]['header'][j]['range'][0]
        t = data_train[i]['annotations'][0]['header'][j]['range'][1]
        temp = data_train[i]['annotations'][0]['header'][j]['name'] + ": " + data_train[i]['header'][s:t]
        ls.append(temp)
    spans.append(ls)
    ls=[]
    for j in range(len(data_train[i]['annotations'][0]['text'])):
        s = data_train[i]['annotations'][0]['text'][j]['range'][0]
        t = data_train[i]['annotations'][0]['text'][j]['range'][1]
        temp = data_train[i]['annotations'][0]['text'][j]['name'] + ": " + data_train[i]['text'][s:t]
        ls.append(temp)
    spans.append(ls)

for i in range(len(data_train)):
  data_train[i]['annotations'][0]['text'] = sorted(data_train[i]['annotations'][0]['text'], key=lambda item:item['range'][0])
  data_train[i]['annotations'][0]['header'] = sorted(data_train[i]['annotations'][0]['header'], key=lambda item:item['range'][0])

ner_tags = []
for i in range(len(data_train)):
  #header
  ner_mahdi_list = []
  o_s, o_e = 0, 0
  for idx, ner in enumerate(data_train[i]['annotations'][0]['header']):
    if (ner['range'][1] <= o_s or ner['range'][0] <= o_s or ner['range'][0] <= o_e or ner['range'][1] <= o_e) and idx !=0:
      continue
    o_e = ner['range'][0]
    if idx == 0:
      if ner['range'][0] != 0 :
        ner_mahdi_list = ner_mahdi_list + o_maker(o_s, o_e, data_train[i]['header'])
      ner_mahdi_list = ner_mahdi_list + ner_maker(ner['range'][0], ner['range'][1], data_train[i], 'header', ner['name'])
    else:
      ner_mahdi_list = ner_mahdi_list + o_maker(o_s, o_e, data_train[i]['header'])
      ner_mahdi_list = ner_mahdi_list + ner_maker(ner['range'][0], ner['range'][1], data_train[i], 'header', ner['name'])
    o_s = ner['range'][1]
  if (len(text_tokens[2*i])>len(ner_mahdi_list)):
    ner_mahdi_list = ner_mahdi_list + (['O']*((len(text_tokens[2*i])-len(ner_mahdi_list))))
  ner_tags.append(ner_mahdi_list)
  #text
  ner_mahdi_list = []
  o_s, o_e = 0, 0
  for idx, ner in enumerate(data_train[i]['annotations'][0]['text']):
    if (ner['range'][1] <= o_s or ner['range'][0] <= o_s or ner['range'][0] <= o_e or ner['range'][1] <= o_e) and idx !=0:
      continue
    o_e = ner['range'][0]
    if idx == 0:
      if ner['range'][0] != 0 :
        ner_mahdi_list = ner_mahdi_list + o_maker(o_s, o_e, data_train[i]['text'])
      ner_mahdi_list = ner_mahdi_list + ner_maker(ner['range'][0], ner['range'][1], data_train[i], 'text', ner['name'])
    else:
      ner_mahdi_list = ner_mahdi_list + o_maker(o_s, o_e, data_train[i]['text'])
      ner_mahdi_list = ner_mahdi_list + ner_maker(ner['range'][0], ner['range'][1], data_train[i], 'text', ner['name'])
    o_s = ner['range'][1]
  if (len(text_tokens[2*i+1])>len(ner_mahdi_list)):
    ner_mahdi_list = ner_mahdi_list + (['O']*((len(text_tokens[2*i+1])-len(ner_mahdi_list))))
  ner_tags.append(ner_mahdi_list)


ner_tags_num = [[labels.index(tag) for tag in ner_tag] for ner_tag in ner_tags]


train = Dataset.from_dict({"langs":langs,"spans":spans,"ner_tags":ner_tags_num,"tokens":text_tokens})

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


  0%|          | 0/2700 [00:00<?, ?it/s]

In [ ]:
#######################################TEST########################################
text = []
for i in range(len(data_test)):
  text.append(data_test[i]['header'])
  text.append(data_test[i]['text'])

text_tokens = [word_tokenize(sent) for sent in tqdm.tqdm_notebook(text)]

langs = copy.deepcopy(text_tokens)
ner_tags = copy.deepcopy(text_tokens)

for i , x in enumerate(text_tokens):
  for j , y in enumerate(x):
    langs[i][j]='fa'

spans=[]
for i in range(len(data_test)):
    ls = []
    for j in range(len(data_test[i]['annotations'][0]['header'])):
        s = data_test[i]['annotations'][0]['header'][j]['range'][0]
        t = data_test[i]['annotations'][0]['header'][j]['range'][1]
        temp = data_test[i]['annotations'][0]['header'][j]['name'] + ": " + data_test[i]['header'][s:t]
        ls.append(temp)
    spans.append(ls)
    ls=[]
    for j in range(len(data_test[i]['annotations'][0]['text'])):
        s = data_test[i]['annotations'][0]['text'][j]['range'][0]
        t = data_test[i]['annotations'][0]['text'][j]['range'][1]
        temp = data_test[i]['annotations'][0]['text'][j]['name'] + ": " + data_test[i]['text'][s:t]
        ls.append(temp)
    spans.append(ls)

for i in range(len(data_test)):
  data_test[i]['annotations'][0]['text'] = sorted(data_test[i]['annotations'][0]['text'], key=lambda item:item['range'][0])
  data_test[i]['annotations'][0]['header'] = sorted(data_test[i]['annotations'][0]['header'], key=lambda item:item['range'][0])

ner_tags = []
for i in range(len(data_test)):
  #header
  ner_mahdi_list = []
  o_s, o_e = 0, 0
  for idx, ner in enumerate(data_test[i]['annotations'][0]['header']):
    if (ner['range'][1] <= o_s or ner['range'][0] <= o_s or ner['range'][0] <= o_e or ner['range'][1] <= o_e) and idx !=0:
      continue
    o_e = ner['range'][0]
    if idx == 0:
      if ner['range'][0] != 0 :
        ner_mahdi_list = ner_mahdi_list + o_maker(o_s, o_e, data_test[i]['header'])
      ner_mahdi_list = ner_mahdi_list + ner_maker(ner['range'][0], ner['range'][1], data_test[i], 'header', ner['name'])
    else:
      ner_mahdi_list = ner_mahdi_list + o_maker(o_s, o_e, data_test[i]['header'])
      ner_mahdi_list = ner_mahdi_list + ner_maker(ner['range'][0], ner['range'][1], data_test[i], 'header', ner['name'])
    o_s = ner['range'][1]
  if (len(text_tokens[2*i])>len(ner_mahdi_list)):
    ner_mahdi_list = ner_mahdi_list + (['O']*((len(text_tokens[2*i])-len(ner_mahdi_list))))
  ner_tags.append(ner_mahdi_list)
  #text
  ner_mahdi_list = []
  o_s, o_e = 0, 0
  for idx, ner in enumerate(data_test[i]['annotations'][0]['text']):
    if (ner['range'][1] <= o_s or ner['range'][0] <= o_s or ner['range'][0] <= o_e or ner['range'][1] <= o_e) and idx !=0:
      continue
    o_e = ner['range'][0]
    if idx == 0:
      if ner['range'][0] != 0 :
        ner_mahdi_list = ner_mahdi_list + o_maker(o_s, o_e, data_test[i]['text'])
      ner_mahdi_list = ner_mahdi_list + ner_maker(ner['range'][0], ner['range'][1], data_test[i], 'text', ner['name'])
    else:
      ner_mahdi_list = ner_mahdi_list + o_maker(o_s, o_e, data_test[i]['text'])
      ner_mahdi_list = ner_mahdi_list + ner_maker(ner['range'][0], ner['range'][1], data_test[i], 'text', ner['name'])
    o_s = ner['range'][1]
  if (len(text_tokens[2*i+1])>len(ner_mahdi_list)):
    ner_mahdi_list = ner_mahdi_list + (['O']*((len(text_tokens[2*i+1])-len(ner_mahdi_list))))
  ner_tags.append(ner_mahdi_list)


ner_tags_num = [[labels.index(tag) for tag in ner_tag] for ner_tag in ner_tags]


test = Dataset.from_dict({"langs":langs,"spans":spans,"ner_tags":ner_tags_num,"tokens":text_tokens})

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


  0%|          | 0/150 [00:00<?, ?it/s]

In [ ]:
#######################################EVAL########################################
text = []
for i in range(len(data_eval)):
  text.append(data_eval[i]['header'])
  text.append(data_eval[i]['text'])

text_tokens = [word_tokenize(sent) for sent in tqdm.tqdm_notebook(text)]

langs = copy.deepcopy(text_tokens)
ner_tags = copy.deepcopy(text_tokens)

for i , x in enumerate(text_tokens):
  for j , y in enumerate(x):
    langs[i][j]='fa'

spans=[]
for i in range(len(data_eval)):
    ls = []
    for j in range(len(data_eval[i]['annotations'][0]['header'])):
        s = data_eval[i]['annotations'][0]['header'][j]['range'][0]
        t = data_eval[i]['annotations'][0]['header'][j]['range'][1]
        temp = data_eval[i]['annotations'][0]['header'][j]['name'] + ": " + data_eval[i]['header'][s:t]
        ls.append(temp)
    spans.append(ls)
    ls=[]
    for j in range(len(data_eval[i]['annotations'][0]['text'])):
        s = data_eval[i]['annotations'][0]['text'][j]['range'][0]
        t = data_eval[i]['annotations'][0]['text'][j]['range'][1]
        temp = data_eval[i]['annotations'][0]['text'][j]['name'] + ": " + data_eval[i]['text'][s:t]
        ls.append(temp)
    spans.append(ls)

for i in range(len(data_eval)):
  data_eval[i]['annotations'][0]['text'] = sorted(data_eval[i]['annotations'][0]['text'], key=lambda item:item['range'][0])
  data_eval[i]['annotations'][0]['header'] = sorted(data_eval[i]['annotations'][0]['header'], key=lambda item:item['range'][0])

ner_tags = []
for i in range(len(data_eval)):
  #header
  ner_mahdi_list = []
  o_s, o_e = 0, 0
  for idx, ner in enumerate(data_eval[i]['annotations'][0]['header']):
    if (ner['range'][1] <= o_s or ner['range'][0] <= o_s or ner['range'][0] <= o_e or ner['range'][1] <= o_e) and idx !=0:
      continue
    o_e = ner['range'][0]
    if idx == 0:
      if ner['range'][0] != 0 :
        ner_mahdi_list = ner_mahdi_list + o_maker(o_s, o_e, data_eval[i]['header'])
      ner_mahdi_list = ner_mahdi_list + ner_maker(ner['range'][0], ner['range'][1], data_eval[i], 'header', ner['name'])
    else:
      ner_mahdi_list = ner_mahdi_list + o_maker(o_s, o_e, data_eval[i]['header'])
      ner_mahdi_list = ner_mahdi_list + ner_maker(ner['range'][0], ner['range'][1], data_eval[i], 'header', ner['name'])
    o_s = ner['range'][1]
  if (len(text_tokens[2*i])>len(ner_mahdi_list)):
    ner_mahdi_list = ner_mahdi_list + (['O']*((len(text_tokens[2*i])-len(ner_mahdi_list))))
  ner_tags.append(ner_mahdi_list)
  #text
  ner_mahdi_list = []
  o_s, o_e = 0, 0
  for idx, ner in enumerate(data_eval[i]['annotations'][0]['text']):
    if (ner['range'][1] <= o_s or ner['range'][0] <= o_s or ner['range'][0] <= o_e or ner['range'][1] <= o_e) and idx !=0:
      continue
    o_e = ner['range'][0]
    if idx == 0:
      if ner['range'][0] != 0 :
        ner_mahdi_list = ner_mahdi_list + o_maker(o_s, o_e, data_eval[i]['text'])
      ner_mahdi_list = ner_mahdi_list + ner_maker(ner['range'][0], ner['range'][1], data_eval[i], 'text', ner['name'])
    else:
      ner_mahdi_list = ner_mahdi_list + o_maker(o_s, o_e, data_eval[i]['text'])
      ner_mahdi_list = ner_mahdi_list + ner_maker(ner['range'][0], ner['range'][1], data_eval[i], 'text', ner['name'])
    o_s = ner['range'][1]
  if (len(text_tokens[2*i+1])>len(ner_mahdi_list)):
    ner_mahdi_list = ner_mahdi_list + (['O']*((len(text_tokens[2*i+1])-len(ner_mahdi_list))))
  ner_tags.append(ner_mahdi_list)


ner_tags_num = [[labels.index(tag) for tag in ner_tag] for ner_tag in ner_tags]


eval = Dataset.from_dict({"langs":langs,"spans":spans,"ner_tags":ner_tags_num,"tokens":text_tokens})

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


  0%|          | 0/150 [00:00<?, ?it/s]

In [ ]:
import datasets

In [ ]:
dataset = datasets.DatasetDict({"train":train,"validation":eval,"test":test})

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['langs', 'spans', 'ner_tags', 'tokens'],
        num_rows: 2700
    })
    validation: Dataset({
        features: ['langs', 'spans', 'ner_tags', 'tokens'],
        num_rows: 150
    })
    test: Dataset({
        features: ['langs', 'spans', 'ner_tags', 'tokens'],
        num_rows: 150
    })
})

In [ ]:
label_names = labels
label_names

['O',
 'B-PER',
 'I-PER',
 'B-LOC',
 'I-LOC',
 'B-NAT',
 'I-NAT',
 'B-EVE',
 'I-EVE',
 'B-TIM',
 'I-TIM',
 'B-DAT',
 'I-DAT',
 'B-ORG',
 'I-ORG',
 'B-mainLOC',
 'I-mainLOC']

In [ ]:
type(dataset['train'])

datasets.arrow_dataset.Dataset

In [ ]:
dataset.column_names

{'test': ['langs', 'spans', 'ner_tags', 'tokens'],
 'train': ['langs', 'spans', 'ner_tags', 'tokens'],
 'validation': ['langs', 'spans', 'ner_tags', 'tokens']}

In [ ]:
dataset.shape

{'test': (150, 4), 'train': (2700, 4), 'validation': (150, 4)}

In [ ]:
dataset['train']

Dataset({
    features: ['langs', 'spans', 'ner_tags', 'tokens'],
    num_rows: 2700
})

## Data Preprocessing

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("HooshvareLab/bert-base-parsbert-ner-uncased")

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["tokens"], padding="max_length", truncation=True, is_split_into_words=True)

In [ ]:
tokenized_datasets_ = dataset.map(tokenize_function, batched=True)

  0%|          | 0/3 [00:00<?, ?ba/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
#Get the values for input_ids, attention_mask, adjusted labels
def tokenize_adjust_labels(all_samples_per_split):
  tokenized_samples = tokenizer.batch_encode_plus(all_samples_per_split["tokens"], is_split_into_words=True, truncation=True)

  total_adjusted_labels = []

  for k in range(0, len(tokenized_samples["input_ids"])):
    prev_wid = -1
    word_ids_list = tokenized_samples.word_ids(batch_index=k)
    existing_label_ids = all_samples_per_split["ner_tags"][k]
    i = -1
    adjusted_label_ids = []

    for word_idx in word_ids_list:
      # Special tokens have a word id that is None. We set the label to -100 so they are automatically
      # ignored in the loss function.
      if(word_idx is None):
        adjusted_label_ids.append(-100)
      elif(word_idx!=prev_wid):
        i = i + 1
        adjusted_label_ids.append(existing_label_ids[i])
        prev_wid = word_idx
      else:
        label_name = label_names[existing_label_ids[i]]
        adjusted_label_ids.append(existing_label_ids[i])

    total_adjusted_labels.append(adjusted_label_ids)

  #add adjusted labels to the tokenized samples
  tokenized_samples["labels"] = total_adjusted_labels
  return tokenized_samples

tokenized_dataset = dataset.map(tokenize_adjust_labels, batched=True, remove_columns=['tokens', 'ner_tags', 'langs', 'spans'])

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 2700
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 150
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 150
    })
})

In [ ]:
from transformers import DataCollatorForTokenClassification
data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
data_collator

DataCollatorForTokenClassification(tokenizer=PreTrainedTokenizerFast(name_or_path='HooshvareLab/bert-base-parsbert-ner-uncased', vocab_size=100000, model_max_len=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}), padding=True, max_length=None, pad_to_multiple_of=None, label_pad_token_id=-100, return_tensors='pt')

## Fine Tuning

In [ ]:
import torch
from torch.utils.data import DataLoader
from transformers import AutoModelForTokenClassification, AdamW

In [ ]:
#check if gpu is present
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [ ]:
model = AutoModelForTokenClassification.from_pretrained("HooshvareLab/bert-base-parsbert-ner-uncased", num_labels=len(label_names), ignore_mismatched_sizes=True, max_position_embeddings=700)
model.to(device)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at HooshvareLab/bert-base-parsbert-ner-uncased and are newly initialized because the shapes did not match:
- bert.embeddings.position_embeddings.weight: found shape torch.Size([512, 768]) in the checkpoint and torch.Size([700, 768]) in the model instantiated
- classifier.weight: found shape torch.Size([21, 768]) in the checkpoint and torch.Size([17, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([21]) in the checkpoint and torch.Size([17]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(100000, 768, padding_idx=0)
      (position_embeddings): Embedding(700, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

In [ ]:
!pip install nervaluate
from nervaluate import Evaluator

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
from datasets import load_metric

def compute_metrics(p):
    predictions, labels = p

    #select predicted index with maximum logit for each token
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_names[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    evaluator = Evaluator(true_labels, true_predictions, tags=['LOC', 'PER' , 'TIM' , 'DAT' , 'EVE' , 'NAT' , 'mainLOC', 'ORG' , 'MON' , 'PCT' ], loader="list")

    results, results_by_tag = evaluator.evaluate()
    return { "results": results,
            "results_by_tag" :results_by_tag
    }

In [ ]:
import numpy as np
from datasets import load_metric
metric = load_metric("seqeval")

def compute_metrics2(p):
    predictions, labels = p

    #select predicted index with maximum logit for each token
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_names[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

- Fine Tuning using Trainer API

In [ ]:
from transformers import TrainingArguments, Trainer

batch_size = 8
logging_steps = len(tokenized_dataset['train']) // batch_size
epochs = 10

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/bert-fine-tune-ner/results",
    num_train_epochs=epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    evaluation_strategy="epoch",
    disable_tqdm=False,
    logging_steps=logging_steps)


In [ ]:
tokenized_dataset["train"]

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 2700
})

In [ ]:
import numpy as np
x = train['ner_tags']
lst = []
for y in x:
  lst = lst + y

lst = np.array(lst)
label , counts = np.unique(np.array(lst) , return_counts=True)
frequence = np.sum(counts) / counts

for i in range(len(frequence)):
  frequence[i] = float(frequence[i])

label_counts = {}
for label , count in zip(label,frequence):
  label_counts[label] = float(count)
print(label_counts)

{0: 1.1665569704255307, 1: 65.16590701914312, 2: 46.47042253521127, 3: 73.95206896551724, 4: 133.5373599003736, 5: 372.97565217391303, 6: 1649.7, 7: 257.765625, 8: 105.59379615952733, 9: 1026.1291866028707, 10: 682.9968152866242, 11: 165.86310904872389, 12: 114.74638844301765, 13: 57.511665325824616, 14: 32.80725103258376, 15: 519.2760290556901, 16: 777.0326086956521}


In [ ]:
weights = list(label_counts.values())

In [ ]:
from torch import nn
class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss_fn = nn.CrossEntropyLoss(weight=torch.tensor(weights).to(device))
        loss = loss_fn(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

In [ ]:
trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
#fine tune using train method
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 2700
  Num Epochs = 10
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 3380


***** Running Evaluation *****
  Num examples = 150
  Batch size = 8
Trainer is attempting to log a value of "{'ent_type': {'correct': 10, 'incorrect': 157, 'partial': 0, 'missed': 538, 'spurious': 803, 'possible': 705, 'actual': 970, 'precision': 0.010309278350515464, 'recall': 0.014184397163120567, 'f1': 0.011940298507462687}, 'partial': {'correct': 39, 'incorrect': 0, 'partial': 128, 'missed': 538, 'spurious': 803, 'possible': 705, 'actual': 970, 'precision': 0.10618556701030928, 'recall': 0.14609929078014183, 'f1': 0.12298507462686567}, 'strict': {'correct': 3, 'incorrect': 164, 'partial': 0, 'missed': 538, 'spurious': 803, 'possible': 705, 'actual': 970, 'precision': 0.003092783505154639, 'recall': 0.00425531914893617, 'f1': 0.0035820895522388056}, 'exact': {'correct': 39, 'incorrect': 128, 'partial': 0, 'missed': 538, 'spurious': 803, 'possible': 705, 'actual': 970, 'precision': 0.04020618556701031, 'recall': 0.05531914893617021, 'f1': 0.04656716417910448}}" of type <class 'dict'

TrainOutput(global_step=3380, training_loss=2.8025705834112222, metrics={'train_runtime': 1259.8334, 'train_samples_per_second': 21.431, 'train_steps_per_second': 2.683, 'total_flos': 2807534616219000.0, 'train_loss': 2.8025705834112222, 'epoch': 10.0})

In [ ]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 150
  Batch size = 8


Trainer is attempting to log a value of "{'ent_type': {'correct': 76, 'incorrect': 309, 'partial': 0, 'missed': 320, 'spurious': 2215, 'possible': 705, 'actual': 2600, 'precision': 0.02923076923076923, 'recall': 0.10780141843971631, 'f1': 0.04599092284417549}, 'partial': {'correct': 89, 'incorrect': 0, 'partial': 296, 'missed': 320, 'spurious': 2215, 'possible': 705, 'actual': 2600, 'precision': 0.09115384615384615, 'recall': 0.33617021276595743, 'f1': 0.14341906202723148}, 'strict': {'correct': 25, 'incorrect': 360, 'partial': 0, 'missed': 320, 'spurious': 2215, 'possible': 705, 'actual': 2600, 'precision': 0.009615384615384616, 'recall': 0.03546099290780142, 'f1': 0.015128593040847202}, 'exact': {'correct': 89, 'incorrect': 296, 'partial': 0, 'missed': 320, 'spurious': 2215, 'possible': 705, 'actual': 2600, 'precision': 0.03423076923076923, 'recall': 0.12624113475177304, 'f1': 0.053857791225416035}}" of type <class 'dict'> for key "eval/results" as a scalar. This invocation of Tensor

{'epoch': 10.0,
 'eval_loss': 2.7280960083007812,
 'eval_results': {'ent_type': {'actual': 2600,
   'correct': 76,
   'f1': 0.04599092284417549,
   'incorrect': 309,
   'missed': 320,
   'partial': 0,
   'possible': 705,
   'precision': 0.02923076923076923,
   'recall': 0.10780141843971631,
   'spurious': 2215},
  'exact': {'actual': 2600,
   'correct': 89,
   'f1': 0.053857791225416035,
   'incorrect': 296,
   'missed': 320,
   'partial': 0,
   'possible': 705,
   'precision': 0.03423076923076923,
   'recall': 0.12624113475177304,
   'spurious': 2215},
  'partial': {'actual': 2600,
   'correct': 89,
   'f1': 0.14341906202723148,
   'incorrect': 0,
   'missed': 320,
   'partial': 296,
   'possible': 705,
   'precision': 0.09115384615384615,
   'recall': 0.33617021276595743,
   'spurious': 2215},
  'strict': {'actual': 2600,
   'correct': 25,
   'f1': 0.015128593040847202,
   'incorrect': 360,
   'missed': 320,
   'partial': 0,
   'possible': 705,
   'precision': 0.009615384615384616,
 

In [ ]:
predictions, labels, _ = trainer.predict(tokenized_dataset["test"])
predictions = np.argmax(predictions, axis=2)
# Remove ignored index (special tokens)
true_predictions = [
    [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_names[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

evaluator = Evaluator(true_labels, true_predictions, tags=['LOC', 'PER' , 'TIM' , 'DAT' , 'EVE' , 'NAT' , 'mainLOC', 'ORG' ], loader="list")

results, results_by_tag = evaluator.evaluate()
results, results_by_tag


***** Running Prediction *****
  Num examples = 150
  Batch size = 8


({'ent_type': {'actual': 2562,
   'correct': 89,
   'f1': 0.05458448328733516,
   'incorrect': 266,
   'missed': 344,
   'partial': 0,
   'possible': 699,
   'precision': 0.034738485558157686,
   'recall': 0.12732474964234622,
   'spurious': 2207},
  'exact': {'actual': 2562,
   'correct': 95,
   'f1': 0.058264336093222936,
   'incorrect': 260,
   'missed': 344,
   'partial': 0,
   'possible': 699,
   'precision': 0.03708040593286495,
   'recall': 0.13590844062947066,
   'spurious': 2207},
  'partial': {'actual': 2562,
   'correct': 95,
   'f1': 0.13799448022079117,
   'incorrect': 0,
   'missed': 344,
   'partial': 260,
   'possible': 699,
   'precision': 0.08782201405152225,
   'recall': 0.3218884120171674,
   'spurious': 2207},
  'strict': {'actual': 2562,
   'correct': 19,
   'f1': 0.01165286721864459,
   'incorrect': 336,
   'missed': 344,
   'partial': 0,
   'possible': 699,
   'precision': 0.00741608118657299,
   'recall': 0.027181688125894134,
   'spurious': 2207}},
 {'DAT': {'

In [ ]:
from transformers import TrainingArguments, Trainer

batch_size = 8
logging_steps = len(tokenized_dataset['train']) // batch_size
epochs = 1

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/bert-fine-tune-ner/results",
    num_train_epochs=epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    evaluation_strategy="epoch",
    disable_tqdm=False,
    logging_steps=logging_steps)

In [ ]:
trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics2
)

In [ ]:
#fine tune using train method
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 2700
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 338


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,2.880300,2.801885,0.001335,0.021277,0.002512,0.006496


***** Running Evaluation *****
  Num examples = 150
  Batch size = 8
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=338, training_loss=2.880730522454843, metrics={'train_runtime': 113.1317, 'train_samples_per_second': 23.866, 'train_steps_per_second': 2.988, 'total_flos': 280817773841088.0, 'train_loss': 2.880730522454843, 'epoch': 1.0})

In [ ]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 150
  Batch size = 8


/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'epoch': 1.0,
 'eval_accuracy': 0.006496395835187328,
 'eval_f1': 0.002512142019762184,
 'eval_loss': 2.801884651184082,
 'eval_precision': 0.0013348758565453413,
 'eval_recall': 0.02127659574468085,
 'eval_runtime': 1.9471,
 'eval_samples_per_second': 77.038,
 'eval_steps_per_second': 9.758}

In [ ]:
predictions, labels, _ = trainer.predict(tokenized_dataset["test"])
predictions = np.argmax(predictions, axis=2)
# Remove ignored index (special tokens)
true_predictions = [
    [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_names[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
results = metric.compute(predictions=true_predictions, references=true_labels)
results

***** Running Prediction *****
  Num examples = 150
  Batch size = 8


/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'DAT': {'f1': 0.0015644011819920044,
  'number': 64,
  'precision': 0.0007865757734661773,
  'recall': 0.140625},
 'EVE': {'f1': 0.0, 'number': 67, 'precision': 0.0, 'recall': 0.0},
 'LOC': {'f1': 0.0, 'number': 127, 'precision': 0.0, 'recall': 0.0},
 'NAT': {'f1': 0.0, 'number': 23, 'precision': 0.0, 'recall': 0.0},
 'ORG': {'f1': 0.0, 'number': 170, 'precision': 0.0, 'recall': 0.0},
 'PER': {'f1': 0.0, 'number': 204, 'precision': 0.0, 'recall': 0.0},
 'TIM': {'f1': 0.0, 'number': 12, 'precision': 0.0, 'recall': 0.0},
 'mainLOC': {'f1': 0.016546018614270942,
  'number': 32,
  'precision': 0.008556149732620321,
  'recall': 0.25},
 'overall_accuracy': 0.005736446634887291,
 'overall_f1': 0.0026001835423676966,
 'overall_precision': 0.0013735153914518866,
 'overall_recall': 0.024320457796852647}

In [ ]:
true = []
pred = []
for x in true_predictions:
  pred = pred + x
for x in true_labels:
  true = true + x

In [ ]:
import numpy as np
from sklearn.metrics import multilabel_confusion_matrix
conf_mtx = multilabel_confusion_matrix(true, pred, labels=label_names)
for i in range(len(label_names)):
  print("confiusion matrix for : ", label_names[i])
  print(conf_mtx[i])
  print("-"*30)

confiusion matrix for :  O
[[ 1638     0]
 [10739     0]]
------------------------------
confiusion matrix for :  B-PER
[[12173     0]
 [  204     0]]
------------------------------
confiusion matrix for :  I-PER
[[12124     0]
 [  253     0]]
------------------------------
confiusion matrix for :  B-LOC
[[12250     0]
 [  127     0]]
------------------------------
confiusion matrix for :  I-LOC
[[12271     0]
 [  106     0]]
------------------------------
confiusion matrix for :  B-NAT
[[12354     0]
 [   23     0]]
------------------------------
confiusion matrix for :  I-NAT
[[12371     0]
 [    6     0]]
------------------------------
confiusion matrix for :  B-EVE
[[12310     0]
 [   67     0]]
------------------------------
confiusion matrix for :  I-EVE
[[12242     0]
 [  135     0]]
------------------------------
confiusion matrix for :  B-TIM
[[12365     0]
 [   12     0]]
------------------------------
confiusion matrix for :  I-TIM
[[12362     0]
 [   15     0]]
------------